In [1]:
from utils.dataloader import *
from utils.sift import *
from utils.helper import *
from utils.classifier import grid_search, get_report
import pandas as pd
from matplotlib import pyplot as plt
import cv2 as cv
from sklearn.model_selection import train_test_split
import pickle
from scipy import stats

In [2]:
RANDOME_STATE= 66
ROOT_PATH, DATA_PATH, OUT_PATH = get_paths()
print(ROOT_PATH)

/Users/moooooo16/Documents/Computer Vision/Project/elpv


In [3]:
train_collection = test_collection = y_train = y_test = None


with open(os.path.join(OUT_PATH, 'sift_X.pkl'), 'rb') as f:
    (train_collection, test_collection) = pickle.load(f)


with open(os.path.join(OUT_PATH, 'sift_y.pkl'), 'rb') as f:
    (y_train, y_test) = pickle.load(f)


In [4]:

ks = [64, 128, 256]
y_train = np.array(y_train)
y_test = np.array(y_test)

for a, b in zip(train_collection.values(), test_collection.values()):
    print(a.shape, b.shape)


print(y_train.shape, y_test.shape)

(1968, 64) (656, 64)
(1968, 128) (656, 128)
(1968, 256) (656, 256)
(1968,) (656,)


# Undersampling method

441
88
42
235
690
133
37
302

In [5]:
from sklearn.svm import SVC

param = {
    'kernel': ['linear', 'rbf'],
    'C' : [0.01, 0.1, 1, 10, 100]
}


In [6]:
total_classes = np.unique(y_train)
logger = my_logger(os.path.join(OUT_PATH, 'logs/undersampling.log'))
logging.warning(f'k,i,j,best_score,,best_clf,mean_train_score,mean_test_score')


clf_collections = []
predictions_collections = []
votes_collections = []
    
for k in ks:
    
    X_train = np.array(train_collection[k])
    X_test = np.array(test_collection[k])
    clfs = []
    predictions = []
    
    for i in total_classes:
        for j in total_classes:

            if j <= i:
                continue
            
            
            postive = np.where(y_train == i)[0]
            negative = np.where(y_train == j)[0]
            
            if len(postive) > len(negative):
                postive = np.random.choice(postive, len(negative), replace=False)
            elif len(postive) < len(negative):
                negative = np.random.choice(negative, len(postive), replace=False)

            
            X_train_selected = X_train[np.concatenate([postive, negative])]
            y_train_selected = y_train[np.concatenate([postive, negative])]
            clf = SVC()
            best = grid_search( X_train_selected, y_train_selected, clf, param, k = k, classes = (i, j), scoring='accuracy', logger = logger)
            clfs.append(best)

    for clf in clfs:
        pred = clf.predict(X_test)
        predictions.append(pred)

    predictions = np.array(predictions)
    votes, counts = stats.mode(predictions, axis=0)
    
    
    acc, conf_mat = get_report(y_test, votes)
    print()
    print(acc)
    print(conf_mat)
    print('-'*100)
    
    predictions_collections.append(predictions)
    votes_collections.append(votes)
    clf_collections.append(clfs)
    
    logger.handlers[0].flush()

            

Class: [0 1] Val Score: 0.75 use {'C': 100, 'kernel': 'rbf'}
Class: [0 2] Val Score: 0.75 use {'C': 0.01, 'kernel': 'linear'}
Class: [0 3] Val Score: 0.84 use {'C': 0.01, 'kernel': 'linear'}
Class: [0 4] Val Score: 1.00 use {'C': 0.01, 'kernel': 'linear'}
Class: [0 5] Val Score: 1.00 use {'C': 10, 'kernel': 'rbf'}
Class: [0 6] Val Score: 0.96 use {'C': 0.01, 'kernel': 'linear'}
Class: [0 7] Val Score: 1.00 use {'C': 0.1, 'kernel': 'linear'}
Class: [1 2] Val Score: 0.69 use {'C': 0.1, 'kernel': 'rbf'}
Class: [1 3] Val Score: 0.84 use {'C': 1, 'kernel': 'rbf'}
Class: [1 4] Val Score: 1.00 use {'C': 1, 'kernel': 'rbf'}
Class: [1 5] Val Score: 0.99 use {'C': 1, 'kernel': 'rbf'}
Class: [1 6] Val Score: 1.00 use {'C': 1, 'kernel': 'rbf'}
Class: [1 7] Val Score: 0.99 use {'C': 0.1, 'kernel': 'linear'}
Class: [2 3] Val Score: 0.78 use {'C': 1, 'kernel': 'rbf'}
Class: [2 4] Val Score: 1.00 use {'C': 10, 'kernel': 'rbf'}
Class: [2 5] Val Score: 1.00 use {'C': 0.01, 'kernel': 'linear'}
Class: [2 